In [ ]:
from functools import partial

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils

from scripts import run
from utils import mapping
from utils import messages as ms

In [ ]:
#use a class to define your input and output in order to have mutable variables
class gfc_viz_IO:
    def __init__(self):
        #set up your inputs
        self.threshold = 30
        
        #set up output
        self.visualization = False
        
gfc_viz_io = gfc_viz_IO()

In [ ]:
#create a map
gfc_m = mapping.init_gfc_map()

In [ ]:
import ipyvuetify as v

#create the output alert 
gfc_viz_output = wf.OutputWidget(ms.GFC_MAP)

#create the button that start your process
gfc_viz_btn = wf.ProcessBtn('Update map')

#create the widgets following ipyvuetify requirements 
gfc_viz_slider = v.Slider(
    label= 'Threshold', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=30,
)

gfc_viz_inputs = [gfc_viz_slider]

#bind the widget to the inputs
wb.bind(gfc_viz_slider, gfc_viz_io, 'threshold', gfc_viz_output)

#create a process tile
id_ = "gfc_map_widget"
title = 'GFC visualization'


#create a custom tile to display the vizualization map without scrolling down 
gfc_viz_inputs = v.Layout(
        _metadata={'mount-id': 'data-input'},
        class_="pa-5",
        row=True,
        align_center=True, 
        children=[
            v.Flex(xs12=True, children=[gfc_viz_slider]),
            v.Flex(xs12=True, children=[gfc_viz_btn]),
            v.Flex(xs12=True, children=[gfc_viz_output]),
        ]
    )

gfc_viz =  v.Layout(
    _metadata={'mount_id': id_},
    xs12=True,
    row=True,
    class_="ma-5 d-inline",
    children=[
        v.Card(
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=[title]),
                v.Layout(
                    row=True,
                    xs12=True,
                    children=[
                        v.Flex(xs12=True, md6=True, children=[gfc_viz_inputs]),
                        v.Flex(class_="pa-5", xs12=True, md6=True, children=[gfc_m])
                    ]
                )    
            ]
        )
    ]
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, m):
    
    #debug
    #asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    #visualization = True
    
    asset = gfc_aoi_IO.assetId
    threshold = gfc_viz_io.threshold
    visualization = gfc_viz_io.visualization
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return utils.toggleLoading(widget)
    if not wb.checkInput(threshold, output, ms.NO_THRESHOLD): return utils.toggleLoading(widget)
    
    #display the results 
    run.displayGfcMap(asset, threshold, m, visualization, output)
    
    #validate the visualization process
    setattr(gfc_viz_io, 'visualization', True)
    
    #toggle the loading button
    return utils.toggleLoading(widget)

gfc_viz_btn.on_event('click', partial(
    process_start, 
    output=gfc_viz_output,
    m=gfc_m,
))

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_viz